# Partie 2 : Prétraitement de textes avec NLTK

Dans ce notebook, nous allons aborder les principales étapes de prétraitement de textes :
- Chargement de données textuelles avec `pandas`
- Nettoyage et normalisation (minuscules, suppression de la ponctuation, etc.)
- Tokenisation
- Suppression des stopwords
- Lemmatisation ou racinisation (stemming)

Nous allons utiliser la bibliothèque **NLTK** (Natural Language Toolkit) pour les opérations de traitement du langage.


In [1]:
# ========================================================================
# 1. Import des bibliothèques et téléchargement des ressources NLTK
# ========================================================================
import pandas as pd
import re
import string

import nltk
# Si nécessaire, décommentez les lignes suivantes pour télécharger les ressources requises
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer

# Définition d'un jeu de stopwords (en anglais par défaut, peut s'adapter à d'autres langues)
stop_words = set(stopwords.words('english'))  

# Instanciation de la classe pour la lemmatisation et le stemming
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

print("Bibliothèques importées et ressources NLTK prêtes.")


Bibliothèques importées et ressources NLTK prêtes.


# 2. Création ou chargement d'un jeu de données textuelles

Pour illustrer la démarche, nous créons un DataFrame minimal de textes factices.
Dans un cas réel, on lirait un fichier CSV ou JSON via `pd.read_csv` ou `pd.read_json`.


In [2]:
# ========================================================================
# 2. Création d'un DataFrame d'exemple
# ========================================================================
data = {
    "id": [1, 2, 3, 4, 5],
    "text": [
        "Hello world! This is a test message.",
        "Cybersecurity is crucial in the modern world.",
        "Visit http://example.com for more info!!!",
        "Buy now!!! Special OFFER, only 10$...",
        "Phishing attacks are on the rise. Stay safe."
    ]
}

df = pd.DataFrame(data)
df


,id,text
0,1,Hello world! This is a test message.
1,2,Cybersecurity is crucial in the modern world.
2,3,Visit http://example.com for more info!!!
3,4,"Buy now!!! Special OFFER, only 10$..."
4,5,Phishing attacks are on the rise. Stay safe.


**Explication** :  
- Nous avons 5 documents textuels, chacun ayant un champ `id` et un champ `text`.  
- Les phrases contiennent divers éléments comme ponctuation, URL, symboles, etc.

---
# 3. Nettoyage et normalisation

Les étapes de nettoyage et de normalisation incluent généralement :
- Passer le texte en minuscules
- Supprimer (ou remplacer) la ponctuation, les caractères spéciaux, les URLs, etc.
- (Optionnel) Supprimer les nombres


In [3]:
# ========================================================================
# 3. Fonction de nettoyage de base
# ========================================================================
def clean_text(text):
    # 1. Passage en minuscules
    text = text.lower()
    
    # 2. Suppression des URLs (exemple simple, peut être amélioré avec une regex plus précise)
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    
    # 3. Suppression de la ponctuation
    # string.punctuation contient : !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
    # On peut y ajouter des symboles spéciaux si besoin
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # 4. Suppression des chiffres (optionnel)
    text = re.sub(r"\d+", '', text)
    
    # On peut également supprimer les espaces multiples
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

# Application de la fonction à la colonne "text"
df['cleaned_text'] = df['text'].apply(clean_text)

df[['id', 'text', 'cleaned_text']]


,id,text,cleaned_text
0,1,Hello world! This is a test message.,hello world this is a test message
1,2,Cybersecurity is crucial in the modern world.,cybersecurity is crucial in the modern world
2,3,Visit http://example.com for more info!!!,visit for more info
3,4,"Buy now!!! Special OFFER, only 10$...",buy now special offer only
4,5,Phishing attacks are on the rise. Stay safe.,phishing attacks are on the rise stay safe


Nous observons maintenant la version "nettoyée" de chaque document.  
Les URLs ont été supprimées, la ponctuation enlevée, les chiffres retirés, et tout est en minuscules.

---
# 4. Tokenisation

La tokenisation consiste à découper le texte en unités (tokens), souvent des mots ou parfois des sous-mots.
Ici, nous utilisons `word_tokenize` de NLTK.


In [4]:
# ========================================================================
# 4. Tokenisation
# ========================================================================
def tokenize_text(text):
    tokens = word_tokenize(text)  # Tokenisation via NLTK
    return tokens

df['tokens'] = df['cleaned_text'].apply(tokenize_text)
df[['id', 'cleaned_text', 'tokens']]


,id,cleaned_text,tokens
0,1,hello world this is a test message,"[hello, world, this, is, a, test, message]"
1,2,cybersecurity is crucial in the modern world,"[cybersecurity, is, crucial, in, the, modern, ..."
2,3,visit for more info,"[visit, for, more, info]"
3,4,buy now special offer only,"[buy, now, special, offer, only]"
4,5,phishing attacks are on the rise stay safe,"[phishing, attacks, are, on, the, rise, stay, ..."


On obtient ainsi un tableau de tokens pour chaque phrase nettoyée.

---
# 5. Suppression des stopwords

Les *stopwords* sont des mots très fréquents qui n'apportent généralement pas de valeur sémantique forte 
(par exemple en anglais : "the", "is", "in", "at", etc.).  
NLTK fournit une liste par langue, ici nous utilisons l'anglais.  


In [5]:
# ========================================================================
# 5. Suppression des stopwords
# ========================================================================
def remove_stopwords(token_list):
    filtered_tokens = [token for token in token_list if token not in stop_words]
    return filtered_tokens

df['tokens_no_stop'] = df['tokens'].apply(remove_stopwords)
df[['id', 'tokens', 'tokens_no_stop']]


,id,tokens,tokens_no_stop
0,1,"[hello, world, this, is, a, test, message]","[hello, world, test, message]"
1,2,"[cybersecurity, is, crucial, in, the, modern, ...","[cybersecurity, crucial, modern, world]"
2,3,"[visit, for, more, info]","[visit, info]"
3,4,"[buy, now, special, offer, only]","[buy, special, offer]"
4,5,"[phishing, attacks, are, on, the, rise, stay, ...","[phishing, attacks, rise, stay, safe]"


Nous observons que des mots comme "is", "in", "the", "are", etc. ont disparu.

---
# 6. Lemmatisation ou Stemming

- **Stemming** : on réduit le mot à sa racine (par ex. `attacks` -> `attack`), parfois au risque d'être agressif (ex. `studies` -> `studi`).
- **Lemmatisation** : on ramène le mot à sa forme canonique en tenant compte de la partie du discours (ex. `better` -> `good`, `attacks` -> `attack`).

Ici, nous allons illustrer la lemmatisation avec `WordNetLemmatizer` et le stemming avec `PorterStemmer`.


In [6]:
# ========================================================================
# 6. Lemmatisation et Stemming
# ========================================================================

def lemmatize_tokens(token_list):
    # Par défaut, WordNetLemmatizer lemmatise en supposant une partie du discours "n" (nom)
    # Pour un meilleur résultat, on peut faire un POS-tagging, puis lemmatiser en fonction du tag.
    lemmatized = [lemmatizer.lemmatize(token) for token in token_list]
    return lemmatized

def stem_tokens(token_list):
    stemmed = [stemmer.stem(token) for token in token_list]
    return stemmed

df['tokens_lemmatized'] = df['tokens_no_stop'].apply(lemmatize_tokens)
df['tokens_stemmed'] = df['tokens_no_stop'].apply(stem_tokens)

df[['id', 'tokens_no_stop', 'tokens_lemmatized', 'tokens_stemmed']]


,id,tokens_no_stop,tokens_lemmatized,tokens_stemmed
0,1,"[hello, world, test, message]","[hello, world, test, message]","[hello, world, test, messag]"
1,2,"[cybersecurity, crucial, modern, world]","[cybersecurity, crucial, modern, world]","[cybersecur, crucial, modern, world]"
2,3,"[visit, info]","[visit, info]","[visit, info]"
3,4,"[buy, special, offer]","[buy, special, offer]","[buy, special, offer]"
4,5,"[phishing, attacks, rise, stay, safe]","[phishing, attack, rise, stay, safe]","[phish, attack, rise, stay, safe]"


Vous pouvez comparer la différence entre la forme "lemmatisée" et la forme "racinisée" (stemmée) pour chaque token.

---
# 7. Conclusion

Nous avons illustré les principales étapes de prétraitement de textes avec NLTK :

1. Passage en minuscules et nettoyage (suppression de URLs, ponctuation, chiffres, espaces multiples).
2. Tokenisation avec `word_tokenize`.
3. Suppression des *stopwords*.
4. Lemmatisation ou racinisation.

Ces opérations constituent les bases du pipeline NLP avant la phase de vectorisation (TF-IDF, embeddings, etc.) 
et de modélisation (par exemple, classification).

Dans un projet réel, on adaptera chacune de ces étapes :
- Regex plus complexes pour traiter certains patterns,
- Liste de stopwords personnalisée,
- Lemmatisation multilingue,
- etc.

Ensuite, vous pourrez intégrer ce prétraitement dans votre pipeline de classification ou d'analyse de texte.
